# DNNの説明性
一般にAIと呼ばれる技術の中でも特にDNN(深層学習)は何故その答えになったのかがブラックボックスになるというデメリットがある。そこで、反実仮想を用いてモデルについて本来ありえない値を入れてシミュレーションをすることでできる限り説明できるようにする手法としてdiceというライブラリを使用する。

## ライブラリのインポート

In [1]:
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier as MLP
import pandas as pd
import numpy as np
import dice_ml

## データの読み込み

In [2]:
df = pd.read_csv("rossi.csv")
df.head()

,week,arrest,fin,age,race,wexp,mar,paro,prio
0,20,1,0,27,1,0,0,1,3
1,17,1,0,18,1,0,0,1,8
2,25,1,0,19,0,1,0,1,13
3,52,0,1,23,1,1,1,1,1
4,52,0,0,19,0,1,0,1,3


## 原因系変数と結果系変数を分ける
ここでは再逮捕を結果系変数とする。なお、説明性についての解釈をしやすくするため今回は原因系変数の前処理(標準化・正規化など)は行わずありのままの値を使用する。

In [3]:
x = df.drop(["arrest"], axis=1)
y = df["arrest"]

## 訓練データとテストデータを分ける

In [4]:
x_train, x_test, y_train, y_test = tts(x, y, test_size=0.2, random_state=1)

## ニューラルネットワークモデルを作成
できるだけ複雑なモデルを作成する

In [5]:
model = MLP(hidden_layer_sizes=(500, 500, 400, 300))

## 適合

In [6]:
model.fit(x_train, y_train)

MLPClassifier(hidden_layer_sizes=(500, 500, 400, 300))

## 予測および精度の検証
モデルについて精度の検証を行うことはモデルが原因系変数でどれだけ結果を説明できているかの指標となるため必ず行う。

In [7]:
y_pred = model.predict(x_test)

In [8]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.88      0.92        69
           1       0.67      0.89      0.76        18

    accuracy                           0.89        87
   macro avg       0.82      0.89      0.84        87
weighted avg       0.91      0.89      0.89        87



## 反実仮想の作成

In [9]:
d = dice_ml.Data(dataframe = pd.concat([x_test, y_test], axis=1),
                 continuous_features = ["week", "age", "prio"], #量的変数を選択
                 outcome_name = "arrest" #結果系変数を選択
                )
m = dice_ml.Model(model=model, backend="sklearn")
exp = dice_ml.Dice(d, m)

In [10]:
pre_counter = x_test.iloc[0:2, :] 
dice_exp = exp.generate_counterfactuals(pre_counter, total_CFs=4, desired_class = "opposite")
dice_exp.visualize_as_dataframe(show_only_changes=True)

100%|███████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.84it/s]

Query instance (original outcome : 0)


,week,fin,age,race,wexp,mar,paro,prio,arrest
0,52,0,20,1,0,0,1,9,0



Diverse Counterfactual set (new outcome: 1.0)


,week,fin,age,race,wexp,mar,paro,prio,arrest
0,43,0,27,1,0,0,1,-,1
1,13,0,-,1,0,1,1,-,1
2,44,0,-,1,1,0,1,-,1
3,45,0,-,1,1,0,1,-,1


Query instance (original outcome : 1)


,week,fin,age,race,wexp,mar,paro,prio,arrest
0,36,1,23,1,0,0,0,3,1



Diverse Counterfactual set (new outcome: 0.0)


,week,fin,age,race,wexp,mar,paro,prio,arrest
0,44,1,17,1,0,0,0,-,0
1,51,1,18,1,0,0,0,-,0
2,44,1,-,1,0,0,0,-,0
3,46,1,21,1,0,0,0,-,0


元のデータ(上)と反実仮想(下)のデータが出力される。ここで、陽性と陰性が逆転するためのデータが指定された個数(total_CFs)だけ表示される。